In [13]:
import time 
import pandas as pd 

from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

In [14]:
# Driver setup
options = webdriver.ChromeOptions() 
options.headless = True #
options.page_load_strategy = 'none' 
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path) 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)

/var/folders/28/3jbrg3jj0n96cgrlgn5h96yw0000gp/T/ipykernel_1659/1037692766.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True #


In [11]:
#Dynamic url based on page number from user input
def get_url(num_page): 
    
    return ("https://games.crossfit.com/leaderboard/open/2023?view=0&division=2&region=0&scaled=1&sort=0"+
            f"&page={num_page}")

#Get the source code of the page
def get_content(engine_driver,url): 
    
    engine_driver.get(url)
    time.sleep(2) #Play with this int upon your internet connection 
    
    return WebDriverWait(engine_driver, 10).until(
        EC.presence_of_element_located((By.XPATH,"//*[@id='leaderboardSponsorVisible']/div/div[2]")))

#Add every athlete country on the page in a list
def get_country_list(source):  
    
    list1=[]
    
    country_flag = source.find_elements(By.TAG_NAME, "img")
    
    for country in range(0,len(country_flag)):
        list1.append(country_flag[country].get_property('attributes')[1]['textContent'])
    return list1

#Add every athlete name on the page in a list        
def get_full_name(source): 
    
    list2=[]
    
    athlete_name = source.find_elements(By.CLASS_NAME, "full-name")
    
    for name in range(0,len(athlete_name),2):
        list2.append(athlete_name[name].text)
    return list2

#Loop through page, collect list on every page, create a dict out of it and export as .csv
def get_data_as_csv(engine_driver,total_page: int,output_file_name: str): 
    list_test1=[]
    list_test2=[]
    try:
        for page in tqdm(range(total_page+1),total=total_page,desc ="Data is being collected"):
            content = get_content(engine_driver,get_url(page))
            list_test1.extend(get_full_name(content))
            list_test2.extend(get_country_list(content))
            
    except:
        print('an error has been detected')
        if not list_test1:
            print('no data available for backup')
        else:
            df = pd.DataFrame({'name':list_test1,'country':list_test2})
            df.to_csv("Backup.csv", index=False) #if an error occurs, there is a back-up output
            
    else:    
        df = pd.DataFrame({'name':list_test1,'country':list_test2})
        df.to_csv(f"{output_file_name}.csv", index=False)

In [15]:
#get_data_as_csv(driver,1,'women-rx')